In [1]:
x = 2
b = 1

In [2]:
x + b

3

In [7]:
import torch
import numpy

In [10]:
xt = torch.tensor(2.0, requires_grad=True)
bt = torch.tensor(1.0)

In [11]:
y = xt + bt; y

tensor(3., grad_fn=<AddBackward0>)

In [12]:
y.backward()

In [13]:
xt.grad

tensor(1.)

In [90]:
import pdb

class Value:
    def __init__(self, v, left=None, right=None):
        self.v = v
        self.left = left
        self.right = right
        self.g = None
        self.out = None
    
    def __add__(self, value):
        result = self.v + getattr(value, "v", value)
        return self._forward(result, "+", value)
    
    def __radd__(self, value):
        return self.__radd__(value)
    
    def __mul__(self, value):
        result = self.v * getattr(value, "v", value)
        return self._forward(result, "*", value)
    
    def __rmul__(self, value):
        return self.__mul__(value)
    
    def __pow__(self, value):
        result = self.v ** getattr(value, "v", value)
        return self._forward(result, "pow", value)
    
    def __sub__(self, value):
        return self + -1 * value
    
    def __rsub__(self, value):
        return value + -1 * self
    
    def __truediv__(self, value):
        return self * value ** -1
    
    def __rtruediv__(self, value):
        return value * self ** -1
    
    def _forward(self, result, op, other):
        self.op = op
        self.inp = getattr(other, "v", other)
        self.out = Value(result, self, other if isinstance(other, Value) else None)
        return self.out

    def __repr__(self) -> str:
        return str(self.v)
    
    def _calc_grad(self):
        if self.op == "*": return self.inp
        elif self.op == '+': return 1
        elif self.op == 'pow': return self.inp * self.v ** (self.inp - 1)
        else: raise "Operation not implemented"

    def backward(self):
        if self.out is None: self.g = 1
        else: self.g = self.out.g * self._calc_grad()

        if self.right is not None: self.right.backward()
        if self.left is not None: self.left.backward()

xv, xt = Value(4), torch.tensor(4., requires_grad=True)
bv, bt = Value(1), torch.tensor(1., requires_grad=True)

yv = 3 * (2 / xv) ** 2 + bv - 2
yt = 3 * (2 / xt) ** 2 + bt - 2

yv.backward()
yt.backward()

print("yv:", yv, "yt:", yt)
print("xv.grad:", xv.g, "xt.grad:", xt.grad)

assert yv.g == 1
assert yv.v == yt.item()
assert bv.g == bt.grad
assert xv.g == xt.grad

yv: -0.25 yt: tensor(-0.2500, grad_fn=<SubBackward0>)
xv.grad: -0.375 xt.grad: tensor(-0.3750)
